In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import requests
from io import BytesIO
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from tqdm import tqdm  # 進捗バー表示用

# ----------------------
# 1. データの準備 (サンプリング)
# ----------------------
# df = pd.read_csv('your_data.csv') # 実際の読み込み
# ここではデータフレーム df がすでにあると仮定します

# テスト用に「Tシャツ」カテゴリに絞り、ランダムに100件抽出
# ※ category_nameのカラム名は実際のデータに合わせて調整してください
test_df = df[df['item_category_name'] == 'Tシャツ'].sample(n=100, random_state=42).copy()

# インデックスをリセット
test_df = test_df.reset_index(drop=True)
print(f"テスト対象データ数: {len(test_df)}件")

# ----------------------
# 2. モデル準備 (MobileNet V3 Small)
# ----------------------
weights = models.MobileNet_V3_Small_Weights.DEFAULT
model = models.mobilenet_v3_small(weights=weights)
model.classifier = torch.nn.Identity() # 分類層を無効化
model.eval()

# 前処理
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ----------------------
# 3. 特徴量抽出ループ
# ----------------------
features = []
valid_indices = [] # エラーが出なかったインデックスを保存

print("画像の特徴量を抽出中...")
for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
    url = row['material_url']
    try:
        response = requests.get(url, timeout=3)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content)).convert('RGB')
            input_tensor = preprocess(img).unsqueeze(0)
            
            with torch.no_grad():
                feature = model(input_tensor).numpy().flatten()
            
            features.append(feature)
            valid_indices.append(idx)
        else:
            print(f"Skipped (Status {response.status_code}): {url}")
            
    except Exception as e:
        print(f"Error at index {idx}: {e}")

# ----------------------
# 4. 類似度計算
# ----------------------
feature_matrix = np.array(features)
# コサイン類似度行列 (N x N)
similarity_matrix = cosine_similarity(feature_matrix)

# ----------------------
# 5. 結果の可視化 (重要！)
# ----------------------
# ランダムに1つ選んで、似ているTOP3を表示する関数
def visualize_recommendation(query_idx_in_matrix):
    # 元のデータフレーム上のインデックス
    original_idx = valid_indices[query_idx_in_matrix]
    query_item = test_df.iloc[original_idx]
    
    # 類似度スコアを取得
    scores = list(enumerate(similarity_matrix[query_idx_in_matrix]))
    # スコア順にソート (自分自身を除く)
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)[1:4]
    
    # 画像を表示
    fig, axes = plt.subplots(1, 4, figsize=(15, 5))
    
    # クエリ画像（元ネタ）
    try:
        response = requests.get(query_item['material_url'], timeout=3)
        img = Image.open(BytesIO(response.content))
        axes[0].imshow(img)
        axes[0].set_title(f"Query\n{query_item['title'][:10]}...")
        axes[0].axis('off')
    except:
        axes[0].text(0.5, 0.5, 'Image Load Error', ha='center')

    # レコメンド画像 (Top 3)
    for i, (idx_in_matrix, score) in enumerate(sorted_scores):
        rec_idx = valid_indices[idx_in_matrix]
        rec_item = test_df.iloc[rec_idx]
        
        try:
            response = requests.get(rec_item['material_url'], timeout=3)
            img = Image.open(BytesIO(response.content))
            axes[i+1].imshow(img)
            axes[i+1].set_title(f"Top {i+1} (Score: {score:.2f})\n{rec_item['title'][:10]}...")
            axes[i+1].axis('off')
        except:
            axes[i+1].text(0.5, 0.5, 'Image Load Error', ha='center')
            
    plt.tight_layout()
    plt.show()

# テスト実行: ランダムに3パターンほど表示してみる
print("レコメンド結果を表示します...")
import random
for _ in range(3):
    rand_idx = random.randint(0, len(features)-1)
    visualize_recommendation(rand_idx)